In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json

2023-07-31 21:44:02.701765: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

facec = cv2.CascadeClassifier('models/haarcascade_frontalface_default.xml')

# Load the emotion detector model from disk
def load_emotion_model():
    emotion_model = load_model("models/emotion_model_small.h5")
    return emotion_model

model = load_emotion_model()
font = cv2.FONT_HERSHEY_SIMPLEX

class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(0)

    def __del__(self):
        self.video.release()

    # returns camera frames along with bounding boxes and predictions
    def get_frame(self):
        _, fr = self.video.read()
        gray_fr = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
        faces = facec.detectMultiScale(gray_fr, 1.3, 5)

        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]

            roi = cv2.resize(fc, (48, 48))
            roi = roi.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
            roi = np.expand_dims(roi, axis=0)    # Add batch dimension

            # Perform emotion prediction
            pred = model.predict(roi)
            emotion_index = np.argmax(pred)

            emotions_list = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]
            pred_emotion = emotions_list[emotion_index]

            cv2.putText(fr, pred_emotion, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        return fr
        

[ERROR:0@370.495] global persistence.cpp:505 open Can't open file: 'models/haarcascade_frontalface_default.xml' in read mode


ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x151f14d90>.

In [8]:
def gen(camera):
    while True:
        frame = camera.get_frame()
        cv2.imshow('Facial Expression Recognization',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

In [ ]:
gen(VideoCamera())